In [65]:
from game import Game
import numpy as np
import random
import sys
import torch.nn as nn
import torch
import torch.optim as optim
from DQN_CNN import one_hot_encode_game_state, Net


In [60]:
!pip install matplotlib==3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 9.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.5
    Uninstalling matplotlib-3.7.5:
      Successfully uninstalled matplotlib-3.7.5

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [61]:
import matplotlib as plt


ImportError: cannot import name 'get_versions' from 'matplotlib._version' (/Users/yiannizavaliagkos/.pyenv/versions/RL_final/lib/python3.8/site-packages/matplotlib/_version.py)

In [2]:
batch_size = 128
n_epoch = 1000
n_eval = 100
gamma = 1
epsilon = 0.2

input_shape = (16, 4, 4)  # Replace with the actual shape of the game state
num_actions = 4  # Replace with the actual number of actions in the game
n_epoch = 1000
n_eval = 100
SEED = 1234
    
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_model(input_shape, num_actions):
    model = Net(input_shape, num_actions).to(device)
    optimizer = optim.Adam(model.parameters())
    criterion = nn.MSELoss().to(device)
    
    return model, optimizer, criterion

model1, optimizer1, criterion1 = get_model(input_shape, num_actions)
model2, optimizer2, criterion2 = get_model(input_shape, num_actions)

losses1 = []
losses2 = []

In [3]:
def choose_best_action(Q, epsilon=0.2):
    Q_np = np.array([q.item() if hasattr(q, 'item') else q for q in Q])
    Q_np[Q_np == -np.inf] = np.nan
    valid_indices = np.where(~np.isnan(Q_np))[0]
    return np.random.choice(valid_indices) if np.random.rand() < epsilon else np.argmax(Q)

In [47]:
def train_game(game, it):
    global losses
    batch1_label, batch1_output = [], []
    batch2_label, batch2_output = [], []
    step = 1
    while True:
        state = one_hot_encode_game_state(game.state()).unsqueeze(0).permute(0, 3, 1, 2).to(device)
        Q1_values = model1(state)
        Q2_values = model2(state)

        Q1_valid_values = [Q1_values[0][a] if game.is_action_available(a) else float('-inf') for a in range(4)]
        Q2_valid_values = [Q2_values[0][a] if game.is_action_available(a) else float('-inf') for a in range(4)]

        Q_valid = np.array(Q1_valid_values) + np.array(Q2_valid_values)
        
        best_action = choose_best_action(Q_valid,epsilon=epsilon)
        reward = game.do_action(best_action)
        new_state = one_hot_encode_game_state(game.state()).unsqueeze(0).permute(0, 3, 1, 2).to(device)

        if np.random.rand() < 0.5:
            model = model1
            optimizer = optimizer1
            criterion = criterion1
            Q_valid_values = Q1_valid_values
            batch_outputs = batch1_output
            batch_labels = batch1_label
            losses = losses1
        else:
            model = model2
            optimizer = optimizer2
            criterion = criterion2
            Q_valid_values = Q2_valid_values
            batch_outputs = batch2_output
            batch_labels = batch2_label
            losses = losses2

        Q_star = Q_valid_values[best_action]
        
        with torch.no_grad():
            Q_next = model(new_state)
            
        batch_outputs.append(Q_star)
        batch_labels.append(reward + gamma * torch.max(Q_next).item())   


        if step % batch_size == 0 or game.game_over():
            if len(batch_labels) == 0:
                continue
            optimizer.zero_grad()
            label_tensor = torch.tensor(batch_labels, dtype=torch.float32).to(device)
            output_tensor = torch.stack(batch_outputs).to(device)  # Stack tensors directly
            batch_outputs.clear()
            batch_labels.clear()
            loss = criterion(output_tensor, label_tensor)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

            if game.game_over():
                print("epoch: {}, game score: {}".format(it, game.score()))
                return
        
        step += 1

In [51]:
def eval_game(game, scores):
    model1.eval()
    model2.eval()

    with torch.no_grad():
        for i in range(n_eval):
            game = Game()
            while not game.game_over():
                state = one_hot_encode_game_state(game.state()).unsqueeze(0).permute(0, 3, 1, 2).to(device)
                Q1_values = model1(state)
                Q2_values = model2(state)

                Q1_valid_values = [Q1_values[0][a] if game.is_action_available(a) else float('-inf') for a in range(4)]
                Q2_valid_values = [Q2_values[0][a] if game.is_action_available(a) else float('-inf') for a in range(4)]

                Q_valid = np.array(Q1_valid_values) + np.array(Q2_valid_values)
                
                best_action = choose_best_action(Q_valid,epsilon=epsilon)


                game.do_action(best_action)
            print("game score: {}".format(game.score()))
            scores.append(game.score())

In [53]:


model1.train()
model2.train()

scores = []
for it in range(n_epoch):
    game = Game()
    train_game(game, it)
eval_game(game,scores)
print("the mean of the score is {}".format(np.mean(scores)))

the mean of the score is 1518.88


In [16]:
game = Game()

[222.81723022460938,
 676.7169799804688,
 393.17718505859375,
 525.2103881835938,
 752.1127319335938,
 555.3692016601562,
 671.1876220703125,
 235.28726196289062,
 154.57728576660156,
 564.7182006835938,
 176.1781463623047,
 213.7006072998047,
 571.7877197265625,
 655.223876953125,
 472.6535949707031,
 152.3745574951172,
 864.0224609375,
 0.18998005986213684,
 247.00523376464844,
 618.7923583984375,
 1113.514404296875,
 109.99542999267578,
 1519.0789794921875,
 541.5696411132812,
 649.6188354492188,
 232.57713317871094,
 164.44471740722656,
 10.139805793762207,
 384.60687255859375,
 140.91261291503906,
 237.54771423339844,
 167.91322326660156,
 643.2980346679688,
 141.84056091308594,
 239.92910766601562,
 323.61907958984375,
 2623.68359375,
 127.0697250366211,
 230.1575469970703,
 562.2716674804688,
 1656.785888671875,
 619.4667358398438,
 292.0289001464844,
 194.56619262695312,
 142.42543029785156,
 533.9774780273438,
 241.34884643554688,
 552.552001953125,
 489.7397155761719,
 70.756

In [73]:
with open('scores.txt', 'w') as f:
    for row in scores:
        f.write(f'{row}\n')

In [74]:
with open('scores.txt', 'r') as f:
    # Read lines and convert each line back into a list of integers
    scores2 = [list(map(int, line.strip().split(','))) for line in f]

True